In [1]:
import numpy as np
import tensorflow as tf

In [2]:
#VGG Block
class VGGBlock(tf.keras.layers.Layer):
    def __init__(self, filters, layers):
        super(VGGBlock, self).__init__()
        self.filters = filters
        self.layers = layers
        self.conv_layers = [tf.keras.layers.Conv2D(filters, 3,
                            padding='same',
                            activation='relu') for _ in range(layers)]
        self.max_pool = tf.keras.layers.MaxPooling2D(2, strides=2)

    def call(self, inputs):
        x = inputs
        for conv in self.conv_layers:
            x = conv(x)
        x = self.max_pool(x)
        return x
#

In [3]:
class VGG19(tf.keras.Model):
  def __init__(self, num_classes, input_shape):
    super(VGG19, self).__init__()
    self.num_classes = num_classes
    self.input_shape = input_shape
    self.block1 = VGGBlock(64, 2)
    self.block2 = VGGBlock(128, 2)
    self.block3 = VGGBlock(256, 4) # if 3 then vgg16
    self.block4 = VGGBlock(512, 4) # if 3 then vgg16
    self.block5 = VGGBlock(512, 4) # if 3 then vgg16
    self.flatten = tf.keras.layers.Flatten()
    self.fc1 = tf.keras.layers.Dense(4096, activation='relu')
    self.fc2 = tf.keras.layers.Dense(4096, activation='relu')
    self.fc3 = tf.keras.layers.Dense(num_classes, activation='softmax')

  def call(self, inputs):
    x = self.block1(inputs)
    x = self.block2(x)
    x = self.block3(x)
    x = self.block4(x)
    x = self.block5(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    return x

  def build_model(self):
    x = tf.keras.layers.Input(shape=self.input_shape)
    return tf.keras.Model(inputs=[x], outputs=self.call(x))




In [4]:
vgg = VGG19(10, (224, 224, 3))
vgg.build_model().summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg_block (VGGBlock)                 │ (None, 112, 112, 64)        │          38,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg_block_1 (VGGBlock)               │ (None, 56, 56, 128)         │         221,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg_block_2 (VGGBlock)               │ (None, 28, 28, 256)         │       2,065,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg_block_3 (VGGBlock)               │ (None, 14, 14, 512)         │       8,259,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg_block_4 (VGGBlock)               │ (None, 7, 7, 512)           │       9,439,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4096)                │     102,764,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │          40,970 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 139,611,210 (532.57 MB)

 Trainable params: 139,611,210 (532.57 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# load cifar
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

#you need to get the validation
x_train  = x_train / 255.0
x_test = x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    return image, label

train_dataset = train_dataset.map(preprocess_image).batch(128)
test_dataset = test_dataset.map(preprocess_image).batch(128)

In [7]:
optim = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.CategoricalCrossentropy()
vgg.compile(optimizer=optim, loss=loss_fn, metrics=['accuracy'])

In [8]:
vgg.fit(train_dataset, epochs=1, validation_data=test_dataset)

391/391 ━━━━━━━━━━━━━━━━━━━━ 1078s 2s/step - accuracy: 0.0966 - loss: 2.3036 - val_accuracy: 0.1000 - val_loss: 2.3026
